In [1]:
import os
import numpy as np
import torch
from PIL import Image
from pycocotools.coco import COCO
print("Setup complete!")

Setup complete!


In [2]:
folder = 'annotations/'
filename = 'instances_train2017.json'
annFile = os.path.join(folder, filename)

In [3]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=15.41s)
creating index...
index created!


In [4]:
# supporting piece of code

catIds = coco.getCatIds(catNms=['bird'])
imgIds = coco.getImgIds(imgIds = [143])
annIds = coco.getAnnIds(imgIds=imgIds, catIds=catIds, iscrowd = None)
anns = coco.loadAnns(annIds)
print(anns[0])

{'segmentation': [[456.99, 313.98, 448.39, 303.23, 439.78, 284.95, 426.88, 255.91, 426.88, 233.33, 429.03, 222.58, 429.03, 203.23, 433.33, 194.62, 460.22, 194.62, 481.72, 192.47, 503.23, 193.55, 516.13, 208.6, 530.11, 230.11, 535.48, 250.54, 547.31, 269.89, 551.61, 284.95, 552.69, 310.75, 544.09, 320.43, 558.06, 329.03, 546.24, 356.99, 535.48, 368.82, 510.75, 349.46, 479.57, 323.66, 470.97, 319.35]], 'area': 14943.291250000002, 'iscrowd': 0, 'image_id': 143, 'bbox': [426.88, 192.47, 131.18, 176.35], 'category_id': 16, 'id': 40595}


In [5]:
class CocoDataset(object):
    def __init__(self, root, transforms, coco):
        self.root = root
        self.transforms = transforms
        self.coco = coco
        # load all image files
        self.imgs = list()
        for cat_name in os.listdir(os.path.join(root, "category_imgs")):
            for img_name in os.listdir(os.path.join(root, "category_imgs", cat_name)):
                self.imgs.append(os.path.join(cat_name, img_name))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "category_imgs", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        
        category_name, image_name = os.path.split(self.imgs[idx])
        
        catIds = self.coco.getCatIds(catNms=[category_name])
        imgIds = self.coco.getImgIds(imgIds = [image_name.split(".")[0]])
        annIds = self.coco.getAnnIds(imgIds=imgIds, catIds=catIds, iscrowd = None)
        anns = self.coco.loadAnns(annIds)
        
        num_objs = len(anns)
        boxes = []
        labels = []
        iscrowd = []
        area = []
        for ann in anns:
        # get bounding box coordinates for each mask
            boxes.append(ann["bbox"])
            labels.append(ann['category_id'])
            iscrowd.append(ann['iscrowd'])
            area.append(ann['area'])
        
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.as_tensor(labels, dtype=torch.int64)

        image_id = torch.tensor([idx])
        area = torch.as_tensor(area)
        # suppose all instances are not crowd
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [6]:
import torchvision.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [8]:
dataset = CocoDataset('', get_transform(train=True), coco)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4,
                                          shuffle=True, num_workers=2)

In [9]:
# get some random training images
dataiter = iter(data_loader)
images, labels = dataiter.next()

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mikhail/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 185, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mikhail/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mikhail/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-5-2ffae8b5cc20>", line 54, in __getitem__
    img, target = self.transforms(img, target)
TypeError: __call__() takes 2 positional arguments but 3 were given
